In [1]:
!pip install -r requirements.txt > /dev/null

In [7]:
import csv
import dataclasses
from datetime import datetime
from functools import partial
from pathlib import Path
import typing
from typing import Any, Callable, Dict, List, Union
from urllib.parse import urljoin


import benedict
import pandas as pd
import requests

In [39]:
OWM_API_KEY = "<MY-KEY>" # Path("data/myfreekey.txt").open().read().strip()
STATIONS_CSV_FILE = "data/weather_stations.csv"

In [40]:
station_df = pd.read_csv(STATIONS_CSV_FILE)
station_df[:3]

,id,name,latitude,longitude,altitude
0,7005,ABBEVILLE,50.136000,1.834000,69
1,7015,LILLE-LESQUIN,50.570000,3.097500,47
2,7020,PTE DE LA HAGUE,49.725167,-1.939833,6


In [41]:
resp = requests.get(
    "https://api.openweathermap.org/data/2.5/forecast",
    params=dict(
        appid=OWM_API_KEY,
        lon=10.99, 
        lat=44.34, 
        units="metric",
    ),
)
data = benedict.benedict(resp.json())
data["list[0]"]

{'dt': 1688709600, 'main': {'temp': 17.26, 'feels_like': 17.39, 'temp_min': 16.81, 'temp_max': 17.26, 'pressure': 1021, 'sea_level': 1021, 'grnd_level': 936, 'humidity': 90, 'temp_kf': 0.45}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'clouds': {'all': 40}, 'wind': {'speed': 0.74, 'deg': 317, 'gust': 1.18}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2023-07-07 06:00:00'}

In [10]:
rows = benedict.benedict(resp.json())["list"][:3]
ts2utc = datetime.utcfromtimestamp
[ts2utc(row["dt"]) for row in rows]

[datetime.datetime(2023, 7, 7, 6, 0),
 datetime.datetime(2023, 7, 7, 9, 0),
 datetime.datetime(2023, 7, 7, 12, 0)]

In [11]:
df = pd.DataFrame(
    [dict(forecast_time=ts2utc(row["dt"]), cloudiness=row["clouds.all"], **row["main"]) for row in data["list"]],
)
df[:3]

,forecast_time,cloudiness,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf
0,2023-07-07 06:00:00,50,16.57,16.66,16.57,16.74,1021,1021,936,91,-0.17
1,2023-07-07 09:00:00,50,17.98,18.03,17.98,20.81,1021,1021,938,84,-2.83
2,2023-07-07 12:00:00,51,20.77,20.76,20.77,22.87,1021,1021,938,71,-2.10


In [29]:
@dataclasses.dataclass
class Station:
    statioin_id: int
    name: str
    latitude: str
    longitude: str
    altitude: str

    @staticmethod
    def from_csv(path: Union[str, Path]) -> list["Station"]:
        return [
            Station(*row)
            for row in pd.read_csv(path).values.tolist()
        ]
    
    
stations = Station.from_csv(STATIONS_CSV_FILE)
stations[:3]

[Station(statioin_id=7005, name='ABBEVILLE', latitude=50.136, longitude=1.834, altitude=69),
 Station(statioin_id=7015, name='LILLE-LESQUIN', latitude=50.57, longitude=3.0975, altitude=47),
 Station(statioin_id=7020, name='PTE DE LA HAGUE', latitude=49.725167, longitude=-1.939833, altitude=6)]

In [13]:
import enum
import typing
from typing import Any, Callable, Dict, List, Mapping
import enum
import dataclasses
from dataclasses import dataclass


class IResponse(typing.Protocol):
    def raise_for_status() -> None:
        ...
    
    def json() -> Dict[str, Any]:
        ...
        
        
class IHTTPAgent(typing.Protocol):
    def get(url, params: Dict[str, str]) -> IResponse:
        ...


In [15]:
@dataclass
class WFField:
    field_path: str
    factory: Callable[[Any], Any] = lambda x: x

class WFFields(WFField, enum.Enum):
    forecast_time = ("dt", datetime.utcfromtimestamp)
    temperature =  ("main.temp", float)
    humidity = ("main.humidity", int)
    cloudiness = ("clouds.all", int)

In [17]:
row = benedict.benedict(resp.json())["list[0]"]
{field.name: field.factory(row[field.field_path]) for field in WFFields}

{'forecast_time': datetime.datetime(2023, 7, 7, 6, 0),
 'temperature': 16.57,
 'humidity': 91,
 'cloudiness': 50}

In [20]:
@dataclass
class WFData:
    forecast_time: datetime
    temperature: float
    humidity: int
    cloudiness: int
    
    def _asdict(self):
        return dataclasses.asdict(self)


    @classmethod
    def from_owm(cls, owm_resp: benedict.benedict) -> List["WFData"]:
        return [
            cls(**{field.name: field.factory(row[field.field_path]) for field in WFFields})
            for row in owm_resp["list"]
        ]

In [21]:
WFData.from_owm(benedict.benedict(resp.json()))[:3]

[WFData(forecast_time=datetime.datetime(2023, 7, 7, 6, 0), temperature=16.57, humidity=91, cloudiness=50),
 WFData(forecast_time=datetime.datetime(2023, 7, 7, 9, 0), temperature=17.98, humidity=84, cloudiness=50),
 WFData(forecast_time=datetime.datetime(2023, 7, 7, 12, 0), temperature=20.77, humidity=71, cloudiness=51)]

In [22]:
class SimpleOWMApi:
    base_url: str = "https://api.openweathermap.org/data/2.5/"
    appid: str
    def __init__(self, appid: str, agent: IHTTPAgent = None):
        self.appid = appid
        self.agent = agent or requests

    @classmethod
    def _url(cls, endpoint: str) -> str:
        return urljoin(cls.base_url, endpoint)
    
    def _get(self, *, endpoint: str, **params) -> Mapping[str, Any]:
        try:
            response = self.agent.get(self._url(endpoint), params={"appid": self.appid, **params})
            response.raise_for_status()
        except requests.HTTPError:
            raise
        else:
            return benedict.benedict(response.json())
    
    def forecast(self, *, lon, lat, units="metric") -> List[WFData]:
        data = self._get(endpoint="forecast", lon=lon, lat=lat, units=units)
        return WFData.from_owm(data)

In [26]:
owm = SimpleOWMApi(OWM_API_KEY)
resp = owm.forecast(lon=10.99, lat=44.34, units="metric")
resp[:3]

[WFData(forecast_time=datetime.datetime(2023, 7, 7, 6, 0), temperature=16.7, humidity=91, cloudiness=43),
 WFData(forecast_time=datetime.datetime(2023, 7, 7, 9, 0), temperature=17.75, humidity=85, cloudiness=37),
 WFData(forecast_time=datetime.datetime(2023, 7, 7, 12, 0), temperature=20.19, humidity=74, cloudiness=46)]